GAMES

In [7]:
import pandas as pd
import json

In [8]:
# Leer el archivo y cargar cada línea como un objeto JSON
with open('output_steam_games.json', 'r', encoding='utf-8') as archivo:
    data = [json.loads(line) for line in archivo]

# Convertir la lista de objetos JSON en un DataFrame
data_games = pd.DataFrame(data)
data_games.drop(data_games.index[0:88310], inplace=True)

In [9]:
# Borrar columnas que no seran necesarias para el analisis posterior
data_games.drop(['app_name','early_access','url','reviews_url','tags','publisher','specs'], axis=1, inplace = True)
data_games.reindex()

,genres,title,release_date,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,4.99,761140,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,Free To Play,643980,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,Free to Play,670290,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,0.99,767400,彼岸领域
88314,NaN,NaN,NaN,2.99,773570,NaN
...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530,Sacada
120442,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,4.99,658870,"xropi,stev3ns"


In [10]:
# sacar solo el año de la columna 'release_date'
data_games['release_date'] = pd.to_datetime(data_games['release_date'], errors='coerce').dt.year


In [11]:
# Borrar nulos por cada columna
data_games = data_games.dropna(subset=['genres'])
data_games = data_games.dropna(subset=['title'])
data_games = data_games.dropna(subset=['release_date'])
data_games = data_games.dropna(subset=['price'])
data_games = data_games.dropna(subset=['id'])
data_games = data_games.dropna(subset=['developer'])

In [12]:
# Cambiar tipo de dato de la columna 'release_date'
data_games['release_date'] = data_games['release_date'].astype(int)
data_games['release_date'] = data_games['release_date'].astype(str)

In [13]:
# Cambiar valores de la columna 'price' donde indique que sea gratuito por el numero '0'
data_games['price'] = data_games['price'].replace({'Free To Play': 0})
data_games['price'] = data_games['price'].replace({'Free to Play': 0})
data_games['price'] = data_games['price'].replace({'Play for Free!': 0})


In [14]:
# convierte todos los valores a numericos y si se encuentran texto se elimina
data_games = data_games[pd.to_numeric(data_games['price'], errors='coerce').notnull()]


In [15]:
# cambia el nombre de la columna 'id' a 'item_id'
data_games = data_games.rename(columns={'id':'item_id'})


In [16]:
# separa en varias columnas la columna 'genres' en un nuevo DataFrame y se concatena con el DataFrame original
data_games1 = data_games['genres'].apply(pd.Series)
data_games2 = pd.concat([data_games, data_games1], axis=1)

In [17]:
# Eliminar la columna 'genres' y tambien las columnas generadas cuando se separó en varias columnas y dejando solo la primera columna ['0']
# solo se consideraron los valores de la columna ['0'] como genero del juego
data_games2.drop(['genres'], axis=1, inplace = True)
data_games2.drop([1,2,3,4,5,6,7,8,9], axis=1, inplace = True)

In [18]:
# se seleccionan las variables con caracteristicas mas importantes a criterio
data_games2 = data_games2[['item_id','title','developer','release_date','price']]


In [19]:
# se eliminnan duplicados en caso de haber
data_games2 = data_games2.drop_duplicates()

In [28]:

# se exporta la tabla como archivo separa por comas de nombre 'games'
data_games2.to_csv('games.csv', index=False)

ITEMS

In [21]:
import ast

In [22]:
# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Ruta del archivo JSON
file_path = 'australian_users_items.json'

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
data_it = pd.DataFrame(data_list)

In [29]:
# se desanidan los datos de la columna ['items']
data_it1 = data_it.explode(['items'])
data_it2 = pd.json_normalize(data_it1['items']).set_index(data_it1['items'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

#se eliminan las columnas que no se consideran relevantes para el posterior analisis
data_it3.drop(['items_count','user_url','item_name','steam_id'], axis=1, inplace=True)

# se eliminan los datos nulos por columnas
data_it3 = data_it3.dropna(subset=['item_id'])
data_it3 = data_it3.dropna(subset=['playtime_forever'])

# se eliminan 2 columnas adicionales
data_it3.drop(['items','playtime_2weeks'], axis=1, inplace=True)

# se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)
data_it3 = data_it3[data_it3['playtime_forever'] != 0]

# conservar las 3 columnas mas importantes para el analisis
data_it3 = data_it3[['user_id','item_id','playtime_forever']]

# se eliminan los duplicados y valores nulos en caso de haber
data_it3 = data_it3.drop_duplicates()
data_it3 = data_it3.dropna()

# se exporta la tabla como archivo separa por comas de nombre 'user_items'
data_it3.to_csv('user_items.csv', index=False)

REVIEWS

In [24]:
from textblob import TextBlob

In [25]:
# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Ruta del archivo JSON
file_path = 'australian_user_reviews.json'

# Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
data_re = pd.DataFrame(data_list)

In [30]:
# se desanidan los datos de la columna ['reviews']
data_re1 = data_re.explode(['reviews'])
data_re2 = data_re1['reviews'].apply(pd.Series)
data_re3 = pd.concat([data_re1, data_re2], axis=1)

# se crea una columna nueva con el año extraido de los valores de la columna 'posted'
data_re3['year_posted'] = data_re3['posted'].str.extract('(\d{4})')

# se eliminan las columnas con poca relevancia para el posterior analisis
data_re3.drop(['reviews','user_url','last_edited','funny','helpful','posted'], axis=1, inplace = True)

# se eliminan valores nulos por colunas
data_re3 = data_re3.dropna(subset=['year_posted'])
data_re3 = data_re3.dropna(subset=['item_id'])
data_re3 = data_re3.dropna(subset=['review'])
data_re3 = data_re3.dropna(subset=['recommend'])

# seleccion de columnas relevantes para el analisis
data_re3 = data_re3[['user_id','item_id','review','recommend','year_posted',0]]

# reemplazar valores booleanos de T/F por 1/0 y cambiar el tipo de dato a 'int'
data_re3['recommend'] = data_re3['recommend'].replace({'False': 0, 'True': 1}).astype(int)

# borrar la ultima columna [0]
del data_re3[data_re3.columns[-1]]

# funcion que analisa el sentimiento con la libreria TextBlob
def get_sentiment(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment < -0.1:
        return 0
    elif sentiment > 0.1:
        return 2
    else:
        return 1

# crea nueva columna 'sentiment_analysis'aplicando la funcion anterior sobre la columna 'review'
data_re3['sentiment_analysis'] = data_re3['review'].apply(get_sentiment)

# eliminar la columna 'review'
data_re3.drop(['review'], axis=1, inplace = True)

# se eliminan los duplicados y valores nulos en caso de haber
data_re3 = data_re3.drop_duplicates()
data_re3 = data_re3.dropna()

# se exporta la tabla como archivo separa por comas de nombre 'user_reviews'
data_re3.to_csv('user_reviews.csv', index=False)

C:\Users\nahue\AppData\Local\Temp\ipykernel_1892\1034787692.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_re3['recommend'] = data_re3['recommend'].replace({'False': 0, 'True': 1}).astype(int)
